# Deep Reinforcement Learning

In [14]:
Group_Member_1 = "Ali Ghasemzadeh 401106339"
Group_Member_2 = "Mohammad Amin Abbasfar 401106211"
Group_Member_3 = "Ali Najar 401102701"

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [1]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [2]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

In [3]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [4]:
env_name = 'CartPole-v1'

# Create an instance of the environment
env = gym.make(env_name,render_mode='rgb_array')

env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample() 
    
    obs, reward, done, _ , _ = env.step(action) 
    
    # render this frame and add to the list of frames
    frames.append(env.render())
    
    if done:
        env.reset()

env.close()
frames = [cv2.resize(frame, ((frame.shape[1]//16)*16,(frame.shape[0]//16)*16)) for frame in frames]
imageio.mimsave('./cartpole.mp4', frames, fps=25)

C:\Users\Haj Ali\miniconda3\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.

You need to implement the convolutional neural network.


In [7]:
class ActorCritic(nn.Module):
    def __init__(self, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        # TODO: define your model here

        # Actor Network
        self.actor = nn.Sequential(
            nn.Linear(4, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=-1)
        )

        # Critic Network
        self.critic = nn.Sequential(
            nn.Linear(4, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        action_probs = self.actor(x)
        value = self.critic(x)
        return action_probs, value

## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [8]:
class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = int(num_episodes)
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # define your actor critic network, its optimzer and the loss function
        self.policy_net = ActorCritic(hidden_size , self.env.action_space.n).to(self.device)
        self.actor_optim = optim.Adam(self.policy_net.actor.parameters(), lr=self.lr)
        self.critic_optim = optim.Adam(self.policy_net.critic.parameters(), lr=self.lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        # choose an action by sampling from the policy distribution
        state = torch.FloatTensor(state).to(self.device)
        action_probs, _ = self.policy_net(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        return action

    def compute_returns(self, rewards):
        # calculate the discounted rewards used in the training process
        returns = []
        R = 0
        for i in range(len(rewards)):
            R = rewards[i] + self.gamma * R
            returns.insert(0, R)
        return torch.tensor(returns).float().to(self.device)

    def train(self):
        episode_rewards = np.array([])
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                # Implement the training loop
                state = self.env.reset()[0]
                log_probs = []
                values = []
                rewards = []
                episode_reward = 0

                for step in range(self.max_steps):
                    
                    action = self.choose_action(state)
                    next_state, reward, done, _ , _ = self.env.step(int(action))
                    
                    action_probs, value = self.policy_net(torch.FloatTensor(state).to(self.device))
                    log_prob = torch.log(action_probs.squeeze(0)[action])
                    log_probs.append(log_prob.unsqueeze(0))
                    values.append(value)
                    rewards.append(reward)
                    episode_reward += reward

                    state = next_state
                    if done:
                        break
                    # first you need to gather log probabilities, state values and rewards from a trajectory
                    # update the 'episode_reward'
                
                episode_rewards = np.append(episode_rewards, episode_reward)
                
                # calculate the discounted rewards
                returns = self.compute_returns(rewards)
                
                # calculate advantage
                advantage = returns - torch.cat(values)
                
                # compute actor and critic losses
                actor_loss = -(torch.cat(log_probs) * advantage.detach()).mean()
                critic_loss = self.critic_loss(torch.cat(values), returns)
                loss = actor_loss + critic_loss
                
                self.actor_optim.zero_grad()
                self.critic_optim.zero_grad()
                
                actor_loss.backward()
                critic_loss.backward()
                self.actor_optim.step()
                self.critic_optim.step()
                # update the model after performing the backward pass

                
                pbar.set_description(f"Episode {episode}, Reward: {episode_reward}")

        self.env.close()
        return episode_rewards

Define the model and set hyperparameters.

In [18]:
env = gym.make(env_name,render_mode='rgb_array')
num_episodes = 500 # TODO
max_steps = 500 # TODO
gamma = 0.9
lr = 1e-2 # TODO
hidden_size = 256 # TODO

a2c_model = A2CAgent(env,num_episodes,max_steps,gamma,lr,hidden_size) # TODO define your model

Train the model.

In [19]:
rewards = a2c_model.train()

Episode 499, Reward: 500.0: 100%|████████████████████████████████████████████████████| 500/500 [03:30<00:00,  2.38it/s]


## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [20]:
env = gym.make(env_name, render_mode='rgb_array')

model = a2c_model
num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# complete the loop and gather the episode reward of each episode as well the frames of the first episode
frames = []
episode_rewards = []

for i in range(num_episodes):
    state = env.reset()[0]
    episode_reward = 0
    
    # TODO 
    # Implement the evaluation loop
    for step in range(model.max_steps):
        action = model.choose_action(state)
        next_state, reward, done, _ , _ = env.step(int(action))
        episode_reward += reward
        state = next_state
        frames.append(env.render())
        if done:
            break
    
    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")
env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")
imageio.mimsave('./test.mp4', frames, fps=60)

Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Episode Reward: 500.0
Average Reward over 10 episodes: 500.0


In [21]:
show_video('./test.mp4')